# What's my watershed?
A tool to help you see the boundaries of your local watershed based on your zip code. Each zip code may traverse multiple watersheds!

Begin by loading in some helper code!

In [1]:
# We have a folder of chunks of reusable code that we're using across different
#  Notebooks. This step goes and gets the relevant code from that folder so we
#  can use it here. (https://github.com/edgi-govdata-archiving/ECHO_modules/tree/watershed-geo)
!pip install git+https://github.com/edgi-govdata-archiving/ECHO_modules
!git clone https://github.com/edgi-govdata-archiving/ECHO-Geo.git &>/dev/null;

# Geopandas is an open source library for working with geographic data using the
#   data structures library "pandas" (common in Python for data processing).
#   (https://geopandas.org/)
!pip install geopandas  &>/dev/null;

# Topojson is an open source library that lets us keep file sizes small when
#   working with geographic data, so the Notebooks can run faster while still
#   working with detailed shapes. (https://github.com/mattijn/topojson)
!pip install topojson &>/dev/null;

# The previous step installed the libraries (explained above); this tells our
#   Notebook how to refer to them.
import geopandas as geopandas
import topojson as tp
import json

# These code blocks come from our folder (https://github.com/edgi-govdata-archiving/ECHO_modules/)
# Each of the files contains a series of function definitions. By running
#   those files here, we make the functions available in this Notebook.
%run ECHO_modules/DataSet.py
%run ECHO_modules/utilities.py
%run ECHO_modules/make_data_sets.py

# When the code block is done loading , the line below lets us know by printing "Done!"
print("Done!")


Done!


Run this cell to create widgets that you can use to indicate your zip code and the size of watershed you want to see. Watersheds are nested units. The USGS specifies watersheds in terms of "hydrologic unit codes" (HUCs) in 8, 10, and 12 digits, with 8 digit HUCs being the biggest sized watersheds and 12 digit HUCs being the smallest.

In [2]:
# Create a widget that can be used to select a zip
zipcode = widgets.Text(
    value="14226",
    description='Zip Code:',
    disabled=False
)
# Create a widget that can be used to select a watershed size
wsize = widgets.RadioButtons(
    options=[8,10,12],
    description='Watershed size:')
# Display the widgets (below)
display(zipcode)
display(wsize)

Text(value='14226', description='Zip Code:')

RadioButtons(description='Watershed size:', options=(8, 10, 12), value=8)

Run this cell to load watershed data.

In [3]:
# In this cell, we are building a query using the SQL language so we can get
#   information from the database about the places we're interested in

# First, find the HUCs in this zip code
with open("ECHO-Geo/ziphuc12.json") as f:
  ziphuc = json.load(f)
huc12s = ziphuc[zipcode.value]
size = wsize.value

# Convert a list of HUC12s to HUC10s or 8s
def hucker(huclist, size):
  hucs=[]
  if size !=12:
    size = 12-size
    for huc in huclist:
      hucs.append(huc[:-size])
    hucs = list(dict.fromkeys(hucs))
  elif size == 12:
    hucs = huc12s
  else:
    print("HUCs should be at 8,10,or 12 digit level")
  #print(hucs) # For debugging
  return hucs

def query(items, table, attribute):
  # Use the selection from the zip-selecting widget from the previous block
  #   to create a list of the watersheds we're interested in
  selection = ' WHERE'
  if (type(items) == list): #watershed case...
    for item in items:
        selection += ' '+attribute+' LIKE \''+item+'\' OR'
    selection = selection[:-3]
  else: #zip case
    selection += ' '+attribute+' = \''+items+'\''
  table = str(table)
  #print(selection) # For debugging
  #print(table) # For debugging

  # Load data from the Stony Brook University mirror of EPA's ECHO database
  # https://gis.stackexchange.com/questions/112057/sql-query-to-have-a-complete-geojson-feature-from-postgis

  # This builds the query by appending details about our request to the variable "sql"
  sql = """
      SELECT jsonb_build_object(
          'type', 'FeatureCollection', 'features', jsonb_agg(features.feature)
      )
      FROM (
          SELECT jsonb_build_object(
              'type', 'Feature','id', gid, 'geometry',
              ST_AsGeoJSON(geom)::jsonb,'properties',
              to_jsonb(inputs) - 'gid' - 'geom'
          ) AS feature
          FROM (
              SELECT *
              FROM 
  """
  sql += table
  sql += selection
  sql += """
          ) inputs
      ) features;
  """

  # The URL here is the location of the database we're requesting from
  url= 'http://portal.gss.stonybrook.edu/echoepa/index2.php?query=' 
  data_location=url+urllib.parse.quote_plus(sql) + '&pg'

  #print(sql) # For debugging
  #print(data_location) # For debugging

  return data_location

# Here, we check to make sure we can read the data we got back
watersheds=None
zip=None
try:
  waterquery = query(hucker(huc12s,size), "wbdhu"+str(size), "huc"+str(size)) # Can also switch to 10 or 8, but keep huc12s as input 
  watersheds = geopandas.read_file(waterquery)
  zipquery = query(zipcode.value, "tl_2019_us_zcta510", "zcta5ce10")
  zip = geopandas.read_file(zipquery)
  print("Data loaded. Now to map it!")
except:
  print('Something went wrong!')

Data loaded. Now to map it!


Map it!

In [4]:
# create the map using a library called Folium (https://github.com/python-visualization/folium)
m = folium.Map()

style = {"w":{'fillColor': '#0099ff', 'color': '#182799', "weight": 1}, 
         "z":{'fillColor': '#FFFF00', 'color': '#996969', "weight": 1}}   

w = folium.GeoJson(
    watersheds,
    name = "Watersheds",
    style_function = lambda x: style["w"]
).add_to(m)
folium.GeoJsonTooltip(fields=["name"]).add_to(w)

z = folium.GeoJson(
    zip,
    name = "Zip Code",
    style_function = lambda x: style["z"]
).add_to(m)
folium.GeoJsonTooltip(fields=["zcta5ce10"]).add_to(z)

# compute boundaries so that the map automatically zooms in
bounds = w.get_bounds()
m.fit_bounds(bounds, padding=0)

# display the map!
display(m)